# Modelagem de Tópicos com LDA em notícias da Folha de São Paulo

A Modelagem de Tópicos com LDA (Latent Dirichlet Allocation) é utilizada para descobrir tópicos latentes em um conjunto de documentos. O LDA é um modelo probabilístico que assume que os documentos são uma mistura de tópicos e que cada palavra em um documento é atribuída a um tópico específico. Antes de aplicar o modelo, é necessário especificar o número de tópicos que se deseja extrair.

In [ ]:
%pip install pyLDAvis==2.1.2
%python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97718 sha256=eb299ef216a06b270da1297197ad8eaba3b6609a96cec8b9b4852945238004cd
  Stored in directory: /root/.cache/pip/wheels/d9/93/d6/16c95da19c32f037fd75135ea152d0df37254c25cd1a8b4b6c
Successfully built pyLDAvis
2023-08-30 11:39:30.598036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 11:39:31.528011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-30 11:39:33.409930: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had n

In [ ]:
import pandas as pd
import numpy  as np

import re
import spacy
import spacy
from spacy.lang.pt.examples import sentences
import nltk
from nltk.corpus import stopwords

import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim

from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors
import math


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df= pd.read_csv('https://gist.githubusercontent.com/issilva5/3140e5b330cbcbd65ea4e4c456e5409a/raw/10e35f40faeaf9b160b0a7842566d1c78a8118e5/gistfile1.txt')
df.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,titulo,conteudo,categoria
0,Emissões de CO2 ficam estáveis pelo 3º ano seg...,"O mundo conseguiu, pelo terceiro ano consecuti...",ambiente
1,Geleira na Groenlândia derrete e pode elevar o...,Uma importante geleira no noroeste da Groenlân...,ambiente
2,Abate de lobos que atacam rebanhos gera discus...,"O governo francês autorizou, na semana passada...",ambiente
3,Uma megausina solar em deserto no Marrocos pre...,O micro-ônibus atravessa um enorme planalto em...,ambiente
4,Contando elefantes: pesquisadores fazem maior ...,"Há dois anos, pesquisadores cruzam os céus afr...",ambiente
5,Meta de plantio de árvores da Olimpíada do Rio...,Para compensar o impacto ambiental da Olimpíad...,ambiente
6,Nova regra sobre demarcação de área indígena a...,A proposta de emenda constitucional que passa ...,ambiente
7,Ativistas do Greenpeace fazem ato contra desma...,Ativistas do Greenpeace protestaram nesta quin...,ambiente
8,Como era o Saara antes de se tornar o maior de...,"O que hoje é o árido, quente e inóspito desert...",ambiente
9,Fenda em plataforma na Antártida cresce e preo...,O rápido avanço da rachadura no quarto maior i...,ambiente


In [ ]:
df.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(4000, 3)

## Etapa 1 - Processamento dos dados

Removendo pontuação e colocando em minúscula:

In [ ]:
# Remove punctuation
df['conteudo_processed'] = df['conteudo'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
df['conteudo_processed'] = df['conteudo_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
df['conteudo_processed'].head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:2: DeprecationWarning: invalid escape sequence '\.'
<>:2: DeprecationWarning: invalid escape sequence '\.'
<ipython-input-5-1a2514913f5e>:2: DeprecationWarning: invalid escape sequence '\.'
  df['conteudo_processed'] = df['conteudo'].map(lambda x: re.sub('[,\.!?]', '', x))


0    o mundo conseguiu pelo terceiro ano consecutiv...
1    uma importante geleira no noroeste da groenlân...
2    o governo francês autorizou na semana passada ...
3    o micro-ônibus atravessa um enorme planalto em...
4    há dois anos pesquisadores cruzam os céus afri...
Name: conteudo_processed, dtype: object

Tokenização:

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = df.conteudo_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['mundo', 'conseguiu', 'pelo', 'terceiro', 'ano', 'consecutivo', 'manter', 'estaveis', 'suas', 'emissoes', 'de', 'gases', 'co', 'em', 'mas', 'esse', 'avanco', 'ainda', 'insuficiente', 'de', 'acordo', 'com', 'um', 'informe', 'divulgado', 'nesta', 'segunda', 'feira', 'durante', 'conferencia']


Bigramas e stopwords:

In [ ]:
# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
stop_words = stopwords.words('portuguese')
# acrescimo de palavras
stop_words = stop_words + ['ainda', 'ha', 'ver', 'ver', 'ficar', 'fazer', 'dizer', 'tambem', 'ate','voce', 'sao', 'pedir', 'pessoa', 'ter', 'querer', 'levar', 'passar', 'kkk', 'estao']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Funções para remover stopwords, criar bigramas, e realizar lematização:

In [ ]:
def remove_stopwords(texts, stop_words):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words, stop_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'pt' model, keeping only tagger component (for efficiency)
nlp = spacy.load("pt_core_news_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized = remove_stopwords(data_lemmatized, stop_words)

print(data_lemmatized[:1][0][:30])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['mundo', 'conseguir', 'terceiro', 'ano_consecutivo', 'manter', 'estavel', 'emissoes', 'avanco', 'insuficiente', 'acordo', 'informe', 'divulgar', 'nesta_segunda', 'feira', 'conferencia', 'cop', 'resultado', 'animador', 'clara', 'inesperar', 'mudanca', 'relacao', 'rapir', 'crescimento', 'emissoes', 'ano', 'decar', 'precedente', 'global', 'carbon']


### Criando corpus e dictionary

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 3), (1, 1), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 3), (20, 1), (21, 1), (22, 1), (23, 1), (24, 3), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


## Etapa 2 - Treinamento de um modelo de LDA

In [ ]:
k = 10 # definindo a quantidade de tópicos

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Construindo o modelo LDA:

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=k,
                                       random_state=100,
                                       chunksize=400,
                                       passes=50,
                                       per_word_topics=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Etapa 3 - Avaliação do modelo

1. **Quais são as top-10 palavras-chaves mais importantes em cada tópico? Na sua opinião, elas têm relação semântica entre si? Caso sim, tente criar rótulos para cada um dos tópicos.**

In [ ]:
lda_model.print_topics(num_words=10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.017*"ano" + 0.013*"aluno" + 0.009*"escola" + 0.008*"professor" + 0.008*"educacao" + 0.007*"curso" + 0.007*"estudante" + 0.007*"universidade" + 0.007*"dever" + 0.006*"grande"'),
 (1,
  '0.010*"poder" + 0.009*"grande" + 0.009*"empresa" + 0.008*"ano" + 0.006*"novo" + 0.005*"usar" + 0.005*"usuario" + 0.004*"servico" + 0.004*"mercado" + 0.004*"produto"'),
 (2,
  '0.019*"saude" + 0.017*"paciente" + 0.017*"estudo" + 0.017*"medico" + 0.017*"poder" + 0.012*"doenca" + 0.011*"tratamento" + 0.010*"cancer" + 0.009*"caso" + 0.009*"pesquisador"'),
 (3,
  '0.010*"israelense" + 0.010*"palestino" + 0.009*"mosquito" + 0.007*"zika" + 0.007*"inseto" + 0.006*"coral" + 0.006*"islamico" + 0.006*"cientista" + 0.006*"lua" + 0.006*"formiga"'),
 (4,
  '0.016*"presidente" + 0.013*"governo" + 0.010*"afirmar" + 0.007*"ministro" + 0.006*"poder" + 0.005*"politico" + 0.005*"empresa" + 0.005*"acordo" + 0.004*"federal" + 0.004*"deputado"'),
 (5,
  '0.012*"ano" + 0.008*"poder" + 0.007*"bom" + 0.005*"dia" + 0.005

In [ ]:
topic_keywords = []
for topic_id in range(10):
    keywords = [word for word, _ in lda_model.show_topic(topic_id)]
    topic_keywords.append({'Tópico': topic_id, 'Palavras-chave': ', '.join(keywords)})

topic_keywords_df = pd.DataFrame(topic_keywords)
pd.options.display.max_colwidth = 150
topic_keywords_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Tópico,Palavras-chave
0,0,"ano, aluno, escola, professor, educacao, curso, estudante, universidade, dever, grande"
1,1,"poder, grande, empresa, ano, novo, usar, usuario, servico, mercado, produto"
2,2,"saude, paciente, estudo, medico, poder, doenca, tratamento, cancer, caso, pesquisador"
3,3,"israelense, palestino, mosquito, zika, inseto, coral, islamico, cientista, lua, formiga"
4,4,"presidente, governo, afirmar, ministro, poder, politico, empresa, acordo, federal, deputado"
5,5,"ano, poder, bom, dia, saber, tempo, bem, casa, vida, primeiro"
6,6,"vestibular, primeira_fase, fuvest, segunda_fase, cursinho, biologio, medicina, calouro, vunesp, fisica_quimico"
7,7,"jogo, ataque, primeiro, pai, jogador, apo, politico, americano, time, brasileiro"
8,8,"dia, cidade, rua, local, casa, partir, show, restaurante, loja, evento"
9,9,"catolico, cristaos, interlago, papa, porcelano, enxame, papar, ead, quarteto, luis_claudio"


Alguns dos tópicos não exibem uma coesão tão evidente, considerando a natureza do modelo não supervisionado. No entanto, há um grau de coerência em certos tópicos, esses:

* tópico 0: relacionado a educação
* tópico 1: relacionado ao mercado tecnológico
* tópico 2: relacionado a medicina
* tópico 3: relacionado ao mundo
* tópico 4: relacionado a política
* tópico 6: relacionado a vestibular
* tópico 7: relacionado a esporte
* tópico 9: relacionado a religião

Por outro lado, as palavras nos tópicos 5 e 8 carecem de uma ligação evidente entre si, tornando complexa a tarefa de criar rótulos coerentes para esses tópicos. No entanto, optarei por classificá-los da seguinte forma:

* tópico 5: relacionado a dia-a-dia
* tópico 8: relacionado a metrópole


In [ ]:
nomes = ['educação', 'mercado tecnológico', 'medicina', 'mundo', 'política', 'dia-a-dia', 'vestibular', 'esporte', 'metrópole', 'religião']
topic_keywords_df['Nome tópico'] = nomes
topic_keywords_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Tópico,Palavras-chave,Nome tópico
0,0,"ano, aluno, escola, professor, educacao, curso, estudante, universidade, dever, grande",educação
1,1,"poder, grande, empresa, ano, novo, usar, usuario, servico, mercado, produto",mercado tecnológico
2,2,"saude, paciente, estudo, medico, poder, doenca, tratamento, cancer, caso, pesquisador",medicina
3,3,"israelense, palestino, mosquito, zika, inseto, coral, islamico, cientista, lua, formiga",mundo
4,4,"presidente, governo, afirmar, ministro, poder, politico, empresa, acordo, federal, deputado",política
5,5,"ano, poder, bom, dia, saber, tempo, bem, casa, vida, primeiro",dia-a-dia
6,6,"vestibular, primeira_fase, fuvest, segunda_fase, cursinho, biologio, medicina, calouro, vunesp, fisica_quimico",vestibular
7,7,"jogo, ataque, primeiro, pai, jogador, apo, politico, americano, time, brasileiro",esporte
8,8,"dia, cidade, rua, local, casa, partir, show, restaurante, loja, evento",metrópole
9,9,"catolico, cristaos, interlago, papa, porcelano, enxame, papar, ead, quarteto, luis_claudio",religião


### Tópico dominante

2. **Analisando a notícia mais representativa por tópico, você acha que os rótulos que você criou para os tópicos fazem sentido? Utilize alguma das notícias como exemplo para validar sua afirmação.**

In [ ]:
# Tópico dominante por documento
def get_dominant_topic_per_document(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_list = []

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_list.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]))
            else:
                break

    sent_topics_df = pd.DataFrame(sent_topics_list)
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents, df.categoria], axis=1)

    df_dominant_topic = sent_topics_df.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'Categoria']
    return df_dominant_topic

# Top-K documentos mais representativo por tópico
def get_topk_documents_per_topic(df_dominant_topic, k = 1):

    sent_topics_sorteddf_mallet = pd.DataFrame()
    sent_topics_outdf_grpd = df_dominant_topic.groupby('Dominant_Topic')

    for i, grp in sent_topics_outdf_grpd:
        sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet,
                                                grp.sort_values(['Topic_Perc_Contrib'], ascending=False).head(k)],
                                                axis=0)

    # Reset Index
    sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

    # Format
    sent_topics_sorteddf_mallet.columns = ['Document_No', 'Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text", 'Categoria']
    return sent_topics_sorteddf_mallet


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df_dominant_topic = get_dominant_topic_per_document(ldamodel=lda_model, corpus=corpus, texts=data_lemmatized)
topk_docs_per_topic = get_topk_documents_per_topic(df_dominant_topic, 3)

pd.options.display.max_colwidth = 100
topk_docs_per_topic.head(40)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Document_No,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text,Categoria
0,1124,0,0.9913,"ano, aluno, escola, professor, educacao, curso, estudante, universidade, dever, grande","[dia, antes, prazo, final, inscricao, enem, numero, candidato, inscrito, exame, chegar, milhoes,...",educacao
1,2302,0,0.9909,"ano, aluno, escola, professor, educacao, curso, estudante, universidade, dever, grande","[greve, servidor, inss, afetar, pagamento, primeira, parcela, salario, aposentado, informar, fei...",mercado
2,878,0,0.9903,"ano, aluno, escola, professor, educacao, curso, estudante, universidade, dever, grande","[partir, janeiro, enem_exame, nacional, ensino_medio, obrigatorio, desejar, acesso, fies_fur, en...",educacao
3,3891,1,0.9946,"poder, grande, empresa, ano, novo, usar, usuario, servico, mercado, produto","[venda, celular, cair, mostrar, recuperacao, trimestre, anterior, dado, consultoria, nesta_terca...",tec
4,3896,1,0.9908,"poder, grande, empresa, ano, novo, usar, usuario, servico, mercado, produto","[comecar, banir, temporariamente, usuario, utilizar, programa, terceiro, personalizar, experienc...",tec
5,437,1,0.9873,"poder, grande, empresa, ano, novo, usar, usuario, servico, mercado, produto","[australiar, tera, corrigir, latitude, longitude, dar, sistema, satelite, organismo, cientifico,...",ciencia
6,1306,2,0.6800,"saude, paciente, estudo, medico, poder, doenca, tratamento, cancer, caso, pesquisador","[malaria, transmitir, picar, mosquito, chance, contrair, doenca, transfusao, sangue, alto, novo,...",equilibrioesaude
7,1505,2,0.5970,"saude, paciente, estudo, medico, poder, doenca, tratamento, cancer, caso, pesquisador","[mulher, sofrer, infeccao, cronico, gengivo, chance, desenvolver, cancer, especialmente, tr, vez...",equilibrioesaude
8,1412,2,0.5757,"saude, paciente, estudo, medico, poder, doenca, tratamento, cancer, caso, pesquisador","[tratamento, melanomo, avancar, tipo, menos, frequente, agressivo, cancer, pele, ganhar, reforco...",equilibrioesaude
9,1876,3,0.8733,"israelense, palestino, mosquito, zika, inseto, coral, islamico, cientista, lua, formiga","[italo, nogueira, pmdb, event, will, develop, wholly, vibir, triggered, political, light, say, e...",esporte


Alguns tópicos não surgiram devido ao maior peso atribuído a um tópico sobre o outro em certos documentos. Agora, em relação a alguns tópicos, a correspondência entre os rótulos e as notícias foi estabelecida corretamente. Isso é evidenciado ao observarmos a categoria desses documentos. Por exemplo, no tópico 0, identificado como 'educação', foram encontrados dois documentos que pertencem à categoria 'educação'. Essa tendência também se manifesta nos tópicos 1, 2, 4 e 7.

### Quantidade de topicos

3. **Qual sua opinião sobre a quantidade de tópicos que foi usada na modelagem? Você acha que a escolha dessa quantidade foi boa? Tente justificar sua resposta usando exemplos, por exemplo, presença de tópicos muito parecidos.**

Em minha perspectiva, empregar 10 tópicos representa um número considerável, uma vez que alguns desses tópicos exibem uma coerência limitada e são de difícil categorização. Isso se evidencia no gráfico a seguir, onde diversos tópicos estão em proximidade tão próxima que poderiam ser unificados em um único tópico

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.211082  0.053155       1        1  29.072809
5      0.185024  0.009116       2        1  23.528303
4      0.215566 -0.005083       3        1  17.819628
0      0.185287  0.050066       4        1  10.376494
7      0.093960 -0.108340       5        1   7.583991
8     -0.026226 -0.259749       6        1   5.229308
2     -0.079903  0.278073       7        1   4.969994
3     -0.276422 -0.004442       8        1   0.898298
9     -0.260615 -0.007831       9        1   0.349632
6     -0.247753 -0.004965      10        1   0.171543, topic_info=             Term         Freq        Total Category  logprob  loglift
298    presidente  2761.000000  2761.000000  Default  30.0000  30.0000
568         poder  6180.000000  6180.000000  Default  29.0000  29.0000
442           dia  3073.000000  3073.000000  Default  28.0000  28.0000
107        estudo  1244.000000  1244.000000  Default  27.0000  27.0000
1568        saude  1007.000000  1007.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
9259      quimica     1.542142     9.415055  Topic10  -6.8485   4.5590
11071  horas_meia     1.606981    13.537280  Topic10  -6.8073   4.2370
7194     biologia     1.461675    13.007745  Topic10  -6.9020   4.1821
4058     sorocaba     1.470094    13.749702  Topic10  -6.8963   4.1324
9627    rio_preto     1.386810    10.166878  Topic10  -6.9546   4.3760

[620 rows x 6 columns], token_table=       Topic      Freq             Term
term                                   
21590      3  0.408848        abstencao
21590      8  0.306636        abstencao
21590     10  0.204424        abstencao
4805       1  0.352646       academicar
4805       9  0.470194       academicar
...      ...       ...              ...
21601     10  0.811406           vunesp
21625     10  0.518297  vunesp_fundacao
36029      8  0.957647          zerkani
9741       8  0.989293             zika
10880      6  0.971086       zona_oeste

[1139 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 5, 1, 8, 9, 3, 4, 10, 7])

### Coerência

4. **Utilizando a métrica de coerência geral e por tópico, faça a sua interpretação dos resultados dos tópicos extraídos.**

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
coherence_lda_per_topic = coherence_model_lda.get_coherence_per_topic()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print('Coerência geral: ', coherence_lda)

print('\nCoerência por tópico: ')
for i in range(10):
  print('tópico ', i, ': ',coherence_lda_per_topic[i])



Coerência geral:  0.48989935052274963

Coerência por tópico: 
tópico  0 :  0.42305751005683073
tópico  1 :  0.35740850863371765
tópico  2 :  0.6540883871963399
tópico  3 :  0.3804346367561971
tópico  4 :  0.4887632270505349
tópico  5 :  0.331763623307993
tópico  6 :  0.4895236179564762
tópico  7 :  0.45668183644481164
tópico  8 :  0.6247893734696648
tópico  9 :  0.6924827843549306


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


A coesão geral não apresenta um valor excessivamente baixo. No que diz respeito aos tópicos, foram observados alguns resultados peculiares. Por exemplo, o tópico 8 exibe uma alta coesão, no entanto, este foi um dos tópicos que encontrei dificuldades em categorizar com base nas palavras.

* Tópico 8 = [dia, cidade, rua, local, casa, partir, show, restaurante, loja, evento]

Por outro lado, os tópicos 2 e 9 exibem as maiores coesões, um resultado com o qual concordo, visto que as palavras estão logicamente interligadas.

* Tópico 2 = [saude, paciente, medico, estudo, doenca, tratamento, cancer, poder, caso, medicamento], apresentando uma relação com medicina e saúde.

* Tópico 9 = [catolico, cristaos, interlago, papa, porcelano, enxame, papar, ead, quarteto, luis_claudio], estabelecendo conexões com a religião.


In [ ]:
pd.options.display.max_colwidth = 150
topic_keywords_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Tópico,Palavras-chave,Nome tópico
0,0,"ano, aluno, escola, professor, educacao, curso, estudante, universidade, dever, grande",educação
1,1,"poder, grande, empresa, ano, novo, usar, usuario, servico, mercado, produto",mercado tecnológico
2,2,"saude, paciente, estudo, medico, poder, doenca, tratamento, cancer, caso, pesquisador",medicina
3,3,"israelense, palestino, mosquito, zika, inseto, coral, islamico, cientista, lua, formiga",mundo
4,4,"presidente, governo, afirmar, ministro, poder, politico, empresa, acordo, federal, deputado",política
5,5,"ano, poder, bom, dia, saber, tempo, bem, casa, vida, primeiro",dia-a-dia
6,6,"vestibular, primeira_fase, fuvest, segunda_fase, cursinho, biologio, medicina, calouro, vunesp, fisica_quimico",vestibular
7,7,"jogo, ataque, primeiro, pai, jogador, apo, politico, americano, time, brasileiro",esporte
8,8,"dia, cidade, rua, local, casa, partir, show, restaurante, loja, evento",metrópole
9,9,"catolico, cristaos, interlago, papa, porcelano, enxame, papar, ead, quarteto, luis_claudio",religião


### Relação entre os tópicos e as categorias

5. **Usando a coluna de categoria dos dados, faça uma análise comparando a distribuição das categorias nos tópicos. As categorias ficaram concentradas em um tópico específico? Você acha que o nome escolhido por você para o tópico faz sentido com a categoria de notícia mais presente nele?**

In [ ]:
pd.options.display.max_colwidth = 150
topic_keywords_df_2 = topic_keywords_df.drop(columns=['Palavras-chave'])
topic_keywords_df_2

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Tópico,Nome tópico
0,0,educação
1,1,mercado tecnológico
2,2,medicina
3,3,mundo
4,4,política
5,5,dia-a-dia
6,6,vestibular
7,7,esporte
8,8,metrópole
9,9,religião


Ao analisarmos os dados abaixo, percebemos que os elementos da categoria 'ambiente' estão sendo predominantemente agrupados no tópico 1, que identifiquei como 'mercado tecnológico' que não casou bem. E nas categorias 'ciência' e 'tec' também estão sendo direcionadas para o tópico 1, ressaltando que ‘tec’ tem uma quantidade alto de documentos indo para o tópico 1.

Por outro lado, as categorias 'mercado', 'mundo' e 'poder' estão principalmente associadas ao tópico 4, denominado 'política'. Esse padrão é particularmente notório na categoria 'poder', em que a grande maioria dos elementos está sendo atribuída ao tópico 4.

Quanto às categorias 'equilibrioesaude', 'esporte' e 'saopaulo', estamos observando que elas estão sendo classificadas no tópico 5, designado como 'dia-a-dia'. Adicionalmente, na categoria 'saopaulo', também ocorre uma associação com o tópico 8, identificado como 'metrópoles'. Porém, na categoria 'equilibrioesaude', não se verificam muitos casos sendo atribuídos ao tópico 2, denominado 'medicina', ficando em terceira posição.

Destaca-se ainda que a categoria 'educacao' apresenta uma presença dominante no tópico 0, intitulado 'educação'


In [ ]:
categoria_topico_contagem = df_dominant_topic.groupby(['Categoria', 'Dominant_Topic']).size().reset_index(name='contagem')
categoria_topico_contagem

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Categoria,Dominant_Topic,contagem
0,ambiente,1,345
1,ambiente,2,1
2,ambiente,4,28
3,ambiente,5,26
4,ciencia,0,22
5,ciencia,1,239
6,ciencia,2,27
7,ciencia,4,8
8,ciencia,5,101
9,ciencia,7,1
